In [1]:
import pandas as pd
import json
import re
import csv
import time
from tqdm import tqdm
from ollama import chat


input_file = '/home/ali/Review_Quality_Benchmark/data/processed/iclr2024_1000_papers.json'
# Load data
df = pd.read_json(input_file)

llm_fields = [
    "llm_Comprehensiveness", "llm_Vagueness", "llm_Objectivity", "llm_Fairness", "llm_Actionability", 
    "llm_Constructiveness", "llm_Relevance Alignment", "llm_Clarity and Readability", "llm_Usage of Technical Terms",
    "llm_Factuality", "llm_Overall Quality", "llm_overall_score_100", "llm_Sentiment Polarity", "llm_Politeness", 
]


# Check for missing fields and add them if not present
for field in llm_fields:
    if field not in df.columns:
        df[field] = pd.NA

# Pattern to extract JSON block
pattern = re.compile(r"<review_assessment>\s*(\{.*?\})\s*</review_assessment>", re.DOTALL)

# Define prompt template
template = """# REVIEW-QUALITY JUDGE

## 0 — ROLE

You are **ReviewInspector-LLM**, a rigorous, impartial meta-reviewer.
Your goal is to assess the quality of a single peer-review against a predefined set of criteria and to provide precise, structured evaluations.

## 1 — INPUTS

Title: {title}
Abstract: {abstract}
Review: {review_text}

## 2 — EVALUATION CRITERIA

Return **only** the scale value or label at right (no rationale text).

| #  | Criterion                    | Allowed scale / label                       | Description                                                                |
| -- | ---------------------------- | ------------------------------------------- | -------------------------------------------------------------------------- |
| 1  | **Comprehensiveness**        | integer **0-5**                             | Extent to which the review covers all key aspects of the paper.            |
| 2  | **Usage of Technical Terms** | integer **0-5**                             | Appropriateness and frequency of domain-specific vocabulary.               |
| 3  | **Factuality**               | **factual / partially factual / unfactual** | Accuracy of the statements made in the review.                             |
| 4  | **Sentiment Polarity**       | **negative / neutral / positive**           | Overall sentiment conveyed by the reviewer.                                |
| 5  | **Politeness**               | **polite / neutral / impolite**             | Tone and manner of the review language.                                    |
| 6  | **Vagueness**                | **none / low / moderate / high / extreme**  | Degree of ambiguity or lack of specificity in the review.                  |
| 7  | **Objectivity**              | integer **0-5**                             | Presence of unbiased, evidence-based commentary.                           |
| 8  | **Fairness**                 | integer **0-5**                             | Perceived impartiality and balance in judgments.                           |
| 9  | **Actionability**            | integer **0-5**                             | Helpfulness of the review in suggesting clear next steps.                  |
| 10 | **Constructiveness**         | integer **0-5**                             | Degree to which the review offers improvements rather than just criticism. |
| 11 | **Relevance Alignment**      | integer **0-5**                             | How well the review relates to the content and scope of the paper.         |
| 12 | **Clarity and Readability**  | integer **0-5**                             | Ease of understanding the review, including grammar and structure.         |
| 13 | **Overall Quality**          | integer **0-100**                           | Holistic evaluation of the review's usefulness and professionalism.        |

## 3 — SCORING GUIDELINES

For 0-5 scales:

* 5 = Outstanding
* 4 = Strong
* 3 = Adequate
* 2 = Weak
* 1 = Very weak
* 0 = Absent/irrelevant

## 4 — ANALYSIS & COMPUTATION (silent)

1. Read and understand the review in the context of the paper title and abstract.
2. Extract quantitative and qualitative signals (e.g., term usage, factual consistency, tone, clarity).
3. Map observations to the corresponding scoring scales.

## 5 — OUTPUT FORMAT (strict)  
Return **exactly one** JSON block wrapped in the tag below — **no comments or extra text**.

```json
<review_assessment>
{{
  "paper_title": "{title}",
  "criteria": {{
    "Comprehensiveness":       ...,
    "Usage of Technical Terms":   ...,
    "Factuality":    ...,
    "Sentiment Polarity":      ...,
    "Politeness":  ...,
    "Vagueness":          ...,
    "Objectivity":             ...,
    "Fairness":         ...,
    "Actionability":        ...,
    "Constructiveness":    ...,
    "Relevance Alignment":    ...,
    "Clarity and Readability":    ...,
    "Relevance Alignment":    ...,
    "Overall Quality":     ...
  }},
  "overall_score_100": ...
}}
</review_assessment>
```
"""

df

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,...,llm_Actionability,llm_Constructiveness,llm_Relevance Alignment,llm_Clarity and Readability,llm_Usage of Technical Terms,llm_Factuality,llm_Overall Quality,llm_overall_score_100,llm_Sentiment Polarity,llm_Politeness
0,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_BcRN,1698598642014,1699636398632,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_hJxN,1698648844616,1699636398538,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_8Cdu,1698863097320,1699636398427,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_Q843,1699416352034,1699636398331,8,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,zt8bb6vC4m,4003,1695329721281,"[~Jianyu_Xu1, ~Yu-Xiang_Wang1]",Pricing with Contextual Elasticity and Heteros...,We study an online contextual dynamic pricing ...,Reviewer_6Reb,1698589805767,1699636362362,6,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,09iOdaeOzp,4531,1695351755791,"[~Mengzhou_Xia1, ~Tianyu_Gao1, ~Zhiyuan_Zeng3,...",Sheared LLaMA: Accelerating Language Model Pre...,"The popularity of LLaMA (Touvron et al., 2023a...",Reviewer_qt4Y,1698822664319,1699636430275,8,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3831,09iOdaeOzp,4531,1695351755791,"[~Mengzhou_Xia1, ~Tianyu_Gao1, ~Zhiyuan_Zeng3,...",Sheared LLaMA: Accelerating Language Model Pre...,"The popularity of LLaMA (Touvron et al., 2023a...",Reviewer_wgpN,1698839708409,1699636430183,5,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3832,030cjlZm4a,9267,1695548092232,"[~Yukti_Makhija1, ~Edward_De_Brouwer1, ~Rahul_...",Learning Predictive Checklists with Probabilis...,Checklists have been widely recognized as effe...,Reviewer_SxKf,1698683826765,1700488544114,8,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3833,030cjlZm4a,9267,1695548092232,"[~Yukti_Makhija1, ~Edward_De_Brouwer1, ~Rahul_...",Learning Predictive Checklists with Probabilis...,Checklists have been widely recognized as effe...,Reviewer_Ucnh,1698794126465,1699637167426,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [2]:
# Process each row
# Set the temperature parameter for the llama model
temperature = 0
seed = 42


# Process each row
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Scoring with LLM"):
    # Skip if all llm fields are already filled
    if all(pd.notna(row.get(field, pd.NA)) for field in llm_fields):
        continue
    # if idx >= 30:
    #     break

    prompt = template.format(
        title=row['submission_title'],
        abstract=row['submission_abstract'],
        review_text=row['total_review']
    )
    
    for attempt in range(5):
        try:
            response = chat("qwen3:8b", messages=[{'role': 'user', 'content': prompt}], options={'temperature': temperature, 'seed': seed})
            content = response['message']['content']
            match = pattern.search(content)
            if not match:
                raise ValueError("No JSON block found")

            parsed = json.loads(match.group(1))
            print(parsed["overall_score_100"])
            for key, val in parsed["criteria"].items():
                df.at[idx, f"llm_{key}"] = val
            df.at[idx, "llm_overall_score_100"] = parsed["overall_score_100"]

            # Save after every successful row
            # df.to_csv(input_file, index=False, quoting=csv.QUOTE_ALL)
            break

        except Exception as e:
            print(f"❌ Error at row {idx}, attempt {attempt + 1}: {e}")

Scoring with LLM:   0%|          | 0/3835 [00:00<?, ?it/s]

Scoring with LLM:   0%|          | 1/3835 [00:14<15:19:25, 14.39s/it]

85


Scoring with LLM:   0%|          | 2/3835 [00:25<13:04:44, 12.28s/it]

65


...

Scoring with LLM: 100%|█████████▉| 3834/3835 [31:07:44<00:16, 16.83s/it]

80


Scoring with LLM: 100%|██████████| 3835/3835 [31:08:02<00:00, 17.16s/it]

Scoring with LLM: 100%|██████████| 3835/3835 [31:08:02<00:00, 29.23s/it]

100


In [3]:
df

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,...,llm_Actionability,llm_Constructiveness,llm_Relevance Alignment,llm_Clarity and Readability,llm_Usage of Technical Terms,llm_Factuality,llm_Overall Quality,llm_overall_score_100,llm_Sentiment Polarity,llm_Politeness
0,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_BcRN,1698598642014,1699636398632,3,...,3,4,5,4,4,factual,85,85,positive,polite
1,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_hJxN,1698648844616,1699636398538,3,...,4,4,5,4,4,partially factual,65,65,neutral,neutral
2,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_8Cdu,1698863097320,1699636398427,3,...,4,4,5,5,5,partially factual,88,88,neutral,polite
3,zxPDdw8koz,4303,1695343783421,"[~Mohammadreza_Salehi3, ~Mehrdad_Farajtabar1, ...",CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,Reviewer_Q843,1699416352034,1699636398331,8,...,4,4,5,4,4,factual,85,85,positive,polite
4,zt8bb6vC4m,4003,1695329721281,"[~Jianyu_Xu1, ~Yu-Xiang_Wang1]",Pricing with Contextual Elasticity and Heteros...,We study an online contextual dynamic pricing ...,Reviewer_6Reb,1698589805767,1699636362362,6,...,5,5,5,4,5,partially factual,85,85,neutral,polite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,09iOdaeOzp,4531,1695351755791,"[~Mengzhou_Xia1, ~Tianyu_Gao1, ~Zhiyuan_Zeng3,...",Sheared LLaMA: Accelerating Language Model Pre...,"The popularity of LLaMA (Touvron et al., 2023a...",Reviewer_qt4Y,1698822664319,1699636430275,8,...,5,5,5,5,5,factual,95,95,positive,polite
3831,09iOdaeOzp,4531,1695351755791,"[~Mengzhou_Xia1, ~Tianyu_Gao1, ~Zhiyuan_Zeng3,...",Sheared LLaMA: Accelerating Language Model Pre...,"The popularity of LLaMA (Touvron et al., 2023a...",Reviewer_wgpN,1698839708409,1699636430183,5,...,4,4,5,4,5,factual,85,85,neutral,polite
3832,030cjlZm4a,9267,1695548092232,"[~Yukti_Makhija1, ~Edward_De_Brouwer1, ~Rahul_...",Learning Predictive Checklists with Probabilis...,Checklists have been widely recognized as effe...,Reviewer_SxKf,1698683826765,1700488544114,8,...,4,4,5,4,5,partially factual,85,85,neutral,polite
3833,030cjlZm4a,9267,1695548092232,"[~Yukti_Makhija1, ~Edward_De_Brouwer1, ~Rahul_...",Learning Predictive Checklists with Probabilis...,Checklists have been widely recognized as effe...,Reviewer_Ucnh,1698794126465,1699637167426,3,...,4,4,5,4,5,partially factual,80,80,neutral,polite


In [4]:
df.to_json('/home/ali/Review_Quality_Benchmark/data/processed/iclr2024_1000_qwen.json', orient='records')

In [5]:
# Filter the first 50 rows and check for rows without null values in "llm_" columns
non_null_count = df.iloc[:][[col for col in df.columns if col.startswith("llm_")]].dropna().shape[0]
print(non_null_count)
print(df.shape)

3826
(3835, 48)
